In [1]:
import os, logging, pickle
from operator import itemgetter
import numpy as np
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
entity_ranking_op_folder = '../../../Data/output/Entity_Ranking'
type_rep_folder = 'TypeRep/type_word'
type_words_folder = 'Type_Words'
entity_rep_folder = 'EntityRep'
entity_context_word_file = 'doc_entity_context_word.p'
type_word_file_name = 'tag_word.p'

word_emnbedding_pretrained_trained_on_corpus = '../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt'

entity_context_word_file = os.path.join(entity_ranking_op_folder, entity_rep_folder, entity_context_word_file)
type_word_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_words_folder, type_word_file_name)

In [3]:
tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others', 'ORG_Accused', 'ORG_Victim', 'ORG_Others', 'PER_Victim', 'PER_Others', 'PER_Accused']
per_tag_list = ['PER_Victim', 'PER_Others', 'PER_Accused']
loc_tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others']
org_tag_list = ['ORG_Accused', 'ORG_Victim', 'ORG_Others']

In [4]:
entity_rep_context_word_file_dict = pickle.load(open(entity_context_word_file, 'rb'))
type_word_dict = pickle.load(open(type_word_file, 'rb'))
#for tag in tag_list:
   # print(tag, type_word_dict[tag], '\n')

In [5]:
word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)

2018-05-01 21:16:17,327 : INFO : loading projection weights from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-05-01 21:18:17,247 : INFO : loaded (400000, 300) matrix from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


KeyError: "word 'Narendra' not in vocabulary"

In [7]:
word_vector = word_vectors.get_vector('was')
size = len(word_vector)
print(size)

300


In [8]:
def group_average(entity, role_tuple):
    num_of_words_in_entity = len(entity)
    num_of_words_in_role = len(role)
    total_number_of_words = num_of_words_in_entity + num_of_words_in_role
    
    sum_of_entity_type_word_vec = np.zeros(size)
    for word in entity:
        try:
            word_vec = word_vectors.get_vector(word)
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
    
    for tup in role_tuple:
        try:
            word_vec = word_vectors.get_vector(tup[0])
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
        
    dot_product = np.dot(sum_of_entity_type_word_vec, sum_of_entity_type_word_vec)
    group_avg = float(dot_product - (total_number_of_words))/(total_number_of_words * (total_number_of_words - 1))
    return group_avg    
    

In [11]:
precision1 = dict()
doc_count_for_tag = dict()

for tag in tag_list:
    doc_count_for_tag[tag] = 0
    precision1[tag] = 0
    

# Precision at K
K = 1
count = 0
for doc_id in entity_rep_context_word_file_dict:
    print(doc_id)
    num_actual_entities_with_role = dict()
    if count > 0:
        break
    per_entities = list()
    loc_entities = list()
    org_entities = list()
    doc_role_dict = entity_rep_context_word_file_dict[doc_id]
    for role in doc_role_dict.keys():
        entities = doc_role_dict[role]
        num_actual_entities_with_role[role] = len(entities)
        if role in per_tag_list:
            for entity in entities:
                per_entities.append((entity, role))
        if role in org_tag_list:
            for entity in entities:
                org_entities.append((entity, role))
        if role in loc_tag_list:
            for entity in entities:
                loc_entities.append((entity, role))
    
    num_per_entity = len(per_entities)
    num_org_entity = len(org_entities)
    num_loc_entity = len(loc_entities)
    
    for role in per_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in per_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_per_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_per_entity)
                
    for role in loc_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in loc_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_loc_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_loc_entity)
                
    for role in org_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in org_entities:
                sim = 2
                sim = group_average(entity[0], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_org_entity)):
                
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_org_entity)
    #print(num_actual_entities_with_role)
    #count = 1

for tag in tag_list:
    if doc_count_for_tag[tag] > 0:
        print(tag, float(precision1[tag] * 100)/doc_count_for_tag[tag], '%')

ev_019_st_002.txt
ev_032_st_002.txt
ev_061_st_004.txt
2010_12_8_st-604.txt
ev_088_st_013.txt
ev_040_st_006.txt
ev_066_st_017.txt
ev_090_st_008.txt
ev_089_st_030.txt
2010_5_2_st-373.txt
ev_094_st_012.txt
ev_064_st_002.txt
2013_6_16_st-42.txt
ev_042_st_003.txt
2013_8_31_st-13.txt
ev_091_st_001.txt
ev_080_st_001.txt
ev_019_st_025.txt
ev_087_st_013.txt
ev_073_st_007.txt
ev_085_st_001.txt
ev_083_st_016.txt
ev_064_st_013.txt
ev_081_st_002.txt
ev_076_st_012.txt
2011_1_21_st-764.txt
ev_062_st_005.txt
ev_085_st_006.txt
2012_5_14_st-381.txt
ev_015_st_001.txt
ev_010_st_001.txt
2013_8_20_st-586.txt
2013_5_23_st-349.txt
ev_082_st_001.txt
ev_086_st_008.txt
ev_064_st_015.txt
ev_049_st_002.txt
ev_093_st_022.txt
ev_083_st_020.txt
ev_019_st_051.txt
ev_074_st_010.txt
ev_039_st_008.txt
ev_086_st_013.txt
ev_050_st_001.txt
ev_014_st_004.txt
ev_093_st_004.txt
ev_058_st_006.txt
ev_074_st_012.txt
ev_037_st_011.txt
2011_12_7_st-2.txt
ev_070_st_006.txt
ev_069_st_004.txt
ev_064_st_022.txt
ev_019_st_022.txt
2013_6

ev_062_st_002.txt
ev_027_st_001.txt
ev_015_st_003.txt
ev_086_st_035.txt
ev_077_st_016.txt
ev_063_st_003.txt
2013_7_8_st-682.txt
ev_035_st_009.txt
ev_066_st_005.txt
ev_079_st_020.txt
2013_9_12_st-773.txt
ev_028_st_003.txt
ev_025_st_004.txt
ev_086_st_019.txt
ev_045_st_003.txt
ev_092_st_023.txt
ev_072_st_019.txt
ev_006_st_001.txt
ev_014_st_001.txt
ev_086_st_024.txt
2013_7_9_st-82.txt
ev_039_st_001.txt
ev_064_st_010.txt
ev_093_st_009.txt
2011_1_19_st-759.txt
ev_088_st_012.txt
ev_084_st_006.txt
ev_039_st_005.txt
ev_058_st_001.txt
ev_017_st_002.txt
ev_083_st_010.txt
2011_7_28_st-255.txt
ev_091_st_005.txt
2010_4_28_st-370.txt
ev_020_st_002.txt
ev_076_st_004.txt
2014_10_9_st-4003.txt
2012_9_20_st-361.txt
ev_084_st_026.txt
ev_016_st_002.txt
2010_9_23_st-549.txt
ev_029_st_005.txt
2012_3_21_st-713.txt
ev_052_st_002.txt
ev_093_st_006.txt
ev_053_st_002.txt
ev_013_st_001.txt
2013_3_8_st-793.txt
ev_002_st_001.txt
ev_086_st_001.txt
ev_085_st_012.txt
ev_063_st_006.txt
2013_4_16_st-27.txt
ev_088_st_010.

ev_065_st_005.txt
ev_045_st_007.txt
ev_089_st_005.txt
ev_064_st_012.txt
2012_10_4_st-750.txt
ev_092_st_011.txt
ev_066_st_018.txt
ev_035_st_005.txt
2013_7_11_st-789.txt
ev_021_st_003.txt
ev_074_st_008.txt
2013_7_3_st-16.txt
ev_070_st_011.txt
ev_091_st_011.txt
ev_086_st_030.txt
ev_072_st_011.txt
ev_089_st_034.txt
ev_060_st_003.txt
ev_063_st_009.txt
ev_032_st_003.txt
2012_2_19_st-641.txt
ev_019_st_033.txt
2013_4_18_st-74.txt
ev_008_st_001.txt
2012_10_18_st-822.txt
ev_059_st_002.txt
ev_014_st_014.txt
ev_028_st_002.txt
ev_067_st_003.txt
ev_087_st_010.txt
ev_067_st_010.txt
ev_093_st_018.txt
ev_075_st_017.txt
ev_082_st_007.txt
ev_064_st_007.txt
ev_090_st_013.txt
2011_7_15_st-11.txt
2014_10_13_st-3.txt
ev_061_st_007.txt
ev_011_st_003.txt
ev_055_st_002.txt
ev_086_st_007.txt
ev_086_st_016.txt
ev_061_st_012.txt
2011_12_8_st-694.txt
2012_3_6_st-74.txt
ev_092_st_012.txt
ev_035_st_015.txt
ev_070_st_005.txt
2012_8_2_st-686.txt
ev_070_st_002.txt
2010_4_29_st-341.txt
ev_026_st_007.txt
2011_9_10_st-144.